# Scraping Elon Musk's latest 200 tweets (user_id,tweet,likes,replies,views,repost)

In [1]:
#Importing necessary libraries for this project
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import scrapy
from scrapy.http import TextResponse
from selenium.webdriver.common.keys import Keys

In [2]:
#Creating a function to extract information
def extract(card):
    #Grabbing username using scrapy library
    username=card.xpath('.//span').css('::text').get()
    
    #Same for user_id,time_uploaded, tweet and overall_stats(replies,repost,likes,bookmarks,views)
    user_id=card.xpath('.//span[contains(text(),"@")]').css('::text').get()
    time_uploaded=card.xpath('.//time').css('::text').get()
    tweet=card.xpath('.//div[@dir="auto" and @class="css-901oao r-18jsvk2 r-1qd0xha r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0" and @data-testid="tweetText"]').css('::text').getall()
    tweet=" ".join(tweet)
    
    #Gonna index the elements in overall_stats as we grabbed every element and we'll assign it to the respective variable.
    overall_stats=card.xpath('.//div[@role="group"]').xpath('@aria-label').get().split(',')
    replies=overall_stats[0].split()[0]
    reposts=overall_stats[1].split()[0]
    likes=overall_stats[2].split()[0]
    bookmarks=overall_stats[3].split()[0]
    try:
        views=overall_stats[4].split()[0]
    except:
        views=0
    if user_id=='@elonmusk':
        return [username,user_id,time_uploaded,tweet,replies,reposts,likes,bookmarks,views]
    else:
        return 'None'

In [3]:
url='https://twitter.com/i/flow/login'
browser=webdriver.Chrome() #Using selenium library's webdriver in order to avoid getting blocked by the website
browser.get(url)
browser.maximize_window()
time.sleep(15)
#grabbing the email element where we'll automate inserting our email.
email_=browser.find_element(By.XPATH,'//input[@autocapitalize="sentences"]')
email_.click()
email_.send_keys('your_email')
time.sleep(3)
#Grabbing next button in order to click it and move to the next stage.
next_1=browser.find_elements(By.XPATH,'//span[@class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"]')
for x in next_1:
    if x.text=='Next':
        click_next=x
    else:
        continue
click_next.click()
time.sleep(4)

#In case it causes error saying no such element and shows unsual activity in the browser and ask to enter our @username and password then uncomment the below code
#And then run it again or let it be

# user_name=browser.find_element(By.XPATH,'//input[@autocapitalize="none"]')
# user_name.click()
# user_name.send_keys('@your_user_name')
# next_2=browser.find_element(By.XPATH,'//span[@class="css-901oao css-16my406 css-1hf3ou5 r-poiln3 r-1inkyih r-rjixqe r-bcqeeo r-qvutc0"]')
# next_2.click()
time.sleep(3)
#Grabbing the password element and entering the password using send keys
password=browser.find_elements(By.XPATH,'//input[@autocapitalize="sentences"]')
password[1].send_keys('your_password') #Enter your password here.
next_3=browser.find_element(By.XPATH,'//span[@class="css-901oao css-16my406 css-1hf3ou5 r-poiln3 r-1inkyih r-rjixqe r-bcqeeo r-qvutc0"]')
next_3.click()
time.sleep(7)
#Grabbing search button and entering Elon musk
search=browser.find_element(By.XPATH,'//input[@placeholder="Search"]')
search.click()
search.send_keys('@elonmusk')
search.send_keys(Keys.ENTER)
time.sleep(3)
elements=browser.find_elements(By.XPATH,'//span[@class="css-901oao css-16my406 r-poiln3 r-bcqeeo r-qvutc0"]')
for x in elements:
    if x.text=='Elon Musk':
        elon=x
elon.click()
time.sleep(10)
#Now we are in elon musk's page and now we'll do the scraping of his tweets only and not the one he reposted
#Creating empty list so we can append elements in them and use it to create a dataframe of it.
username=[]
user_id=[]
time_uploaded=[]
tweet=[]
replies=[]
repost=[]
likes=[]
bookmarks=[]
views=[]
#Using while loops to grab certain amount of tweet records

while len(views)<200:
    if len(views)>200:
        break
    #Using Scrapy response and making selenium elements integrate with scrapy requirements
    response=TextResponse(browser.current_url,body=browser.page_source,encoding="utf-8")
    #Grabbing all elements which has tweets in it
    cards=response.xpath('//article[@data-testid="tweet"]')
    #Using for loops to iterate it and apply our function we created above on every card
    for x in cards:
        new=extract(x)
        #Using if to ignore the none which we make sure to return in extract fucntion if the tweet is not from elon musk.
        if new=='None':
            continue
        #Again using if to avoid appending repeated tweet as we're scrolling down and sometimes after scrolling it grabs above elements
        if new[3] in tweet:
            continue
        else:
            #Appending elements to the above empty lists we created
            username.append(new[0])
            user_id.append(new[1])
            time_uploaded.append(new[2])
            tweet.append(new[3])
            replies.append(new[4])
            repost.append(new[5])
            likes.append(new[6])
            bookmarks.append(new[7])
            views.append(new[8])
    time.sleep(5)
    #Using execute script to scroll down by 0,1500 pixels
    browser.execute_script('window.scrollBy(0,1500)','')
    

In [4]:
#Printing the length of every list to check if they are the same so we don't face any problem converting it to dataframe.
print(len(username),len(user_id),len(time_uploaded),len(tweet),len(replies),len(repost),len(likes),len(bookmarks),len(views))

202 202 202 202 202 202 202 202 202


In [5]:
#Using pandas to create dataframe and appending columns by using lists in which we fed the info above
df=pd.DataFrame(username).rename(columns={0:'Username'})
df['User_id']=user_id
df['Time_uploaded']=time_uploaded
df['Tweet']=tweet
df['Replies']=replies
df['Repost']=repost
df['Likes']=likes
df['Bookmarks']=bookmarks
df['Views']=views
df

,Username,User_id,Time_uploaded,Tweet,Replies,Repost,Likes,Bookmarks,Views
0,Elon Musk,@elonmusk,5h,Very interesting. Hungary is trying hard to ad...,3757,6998,47464,1382,15816448
1,Elon Musk,@elonmusk,23h,Please note that any attempts to manipulate th...,17399,21994,196029,1706,52980697
2,Elon Musk,@elonmusk,Aug 29,Getting ready for next Starship flight,12050,30310,352441,812,44366330
3,Elon Musk,@elonmusk,Aug 27,Dragon docks with @Space_Station .\n\nStrange...,6417,12095,90188,532,57702017
4,Elon Musk,@elonmusk,Aug 26,DOJ needs to sue themselves!,13190,41706,215474,1730,61237294
...,...,...,...,...,...,...,...,...,...
197,Elon Musk,@elonmusk,Jul 12,Practically invisible,25459,28996,412457,1307,34021138
198,Elon Musk,@elonmusk,Jul 11,"Btw, your Tesla can protect you from a bioweap...",8561,11329,107504,967,28434102
199,Elon Musk,@elonmusk,Jul 11,You are free to be your true self here,53495,80921,542427,2849,66230003
200,Elon Musk,@elonmusk,Jul 11,Watching pic in pic video while scrolling this...,10618,12161,162460,560,27475157
